<a href="https://www.kaggle.com/code/ismailmohameddarwish/exploring-european-soccer-database?scriptVersionId=103615857" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Expotatory data analysis #

 In this notebook i will  use the **European Soccer Database** published by **HUGO MATHIEN** in this [link](https://www.kaggle.com/datasets/hugomathien/soccer) to perform some analysis using SQL in extracting useful queries and use python to explore this data and answer some question
 
 ## About database ##
 this data base include the data fot 10 european leaguess from 2009-2016 including matches,teams and players
 it is almost clean data and ready for analysis
 
 

In [ ]:
#importing used libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sqlite3 as sql

# importing data and creating connction to the database to read sqlite queries

path='../input/soccer/database.sqlite'
con=sql.connect(path)

#now we can write sql queries
# explore different tables in the database

sql_master_tables = pd.read_sql('select * from sqlite_master where type="table";',con)
sql_master_tables

So we have 7 tables in this database (as "sqlite_sequence" table is  created and initialized automatically whenever a normal table that contains an AUTOINCREMENT column is created )
so we ignore it

In [ ]:
# now listing  tables name and dropping "sqlite_sequence" table

tablesName = pd.read_sql('select name from sqlite_master where type="table";',con)  #quering tables name
tablesName=tablesName[1:8].squeeze()       #convert the resulting df to series
tablesName=tablesName.values.tolist()      # make a list of tables name
tablesName



In [ ]:
# now we can explore each table and ask some question
Team_Attributes=pd.read_sql('''
select team_long_name,* 
from Team_Attributes
join team 
on team.team_fifa_api_id = Team_Attributes.team_fifa_api_id
''',con)
Team_Attributes.nunique()

In [ ]:
# Team_Attributes=pd.read_sql('select * from Team_Attributes;',con)
# Team_Attributes.drop(Team_Attributes.columns[[1,3]], axis=1, inplace=True)
# Team_Attributes.dropna(axis=1,inplace=True)
# Team_Attributes=Team_Attributes.select_dtypes(include="number")

# Team_Attributes.nunique() # now this dataframe contain team_fifa_api_id with all numeric values 



In [ ]:
Team_Attributes.info()

In [ ]:
Team_Attributes.nunique()

### what the premier league standing in season 2015/2016

In [ ]:
league = pd.read_sql('''
SELECT home.team_long_name AS homeTeam,
       match.home_team_goal,
       (CASE
            WHEN home_team_goal = away_team_goal THEN 1
            WHEN home_team_goal > away_team_goal THEN 3
            WHEN home_team_goal < away_team_goal THEN 0
        END)pointHome,
       away.team_long_name AS awayTeam,
       Match.away_team_goal,
       (CASE
            WHEN home_team_goal = away_team_goal THEN 1
            WHEN home_team_goal > away_team_goal THEN 0
            WHEN home_team_goal < away_team_goal THEN 3
        END)pointAway
FROM Country
JOIN MATCH ON match.country_id = country.id
AND name = "England"
AND season = "2015/2016"
LEFT JOIN Team AS home ON home.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS away ON away.team_api_id = Match.away_team_api_id
;''', con)

league


In [ ]:

league1 = league[['homeTeam', 'home_team_goal', 'pointHome']]

league2 = league[['awayTeam', 'away_team_goal', 'pointAway']]


totalleague = pd.DataFrame(np.concatenate((
    league1.values, league2.values), axis=0))
totalleague.columns = ['team', 'goals', 'points']


leagueSummary = totalleague.groupby('team')[['goals', 'points']].sum()
standing= leagueSummary.sort_values(by='points', ascending=False)

standing

### I can validate my result from this [link](https://www.google.com/search?q=pl+standings+15%2F16&oq=pl+sta&aqs=chrome.0.69i59j69i57j0i271.10502j0j7&sourceid=chrome&ie=UTF-8#sie=lg;/m/012vp1qy;2;/m/02_tc;st;fp;1;;;)




